In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Chain of Thoughts

Chain-of-thought özünde bir kütüphane özelliği değil, prompt tasarımı yani “prompt engineering” ile elde ettiğimiz bir davranış kalıbıdır. Yukarıdaki örnekte ek bir kütüphane ya da özel bir API çağrısı kullanmadık; sadece soruyu sormadan önce prompt’a “Answer the following question step by step. Let’s think step by step.” cümlesini ekledik. Model bu yönlendirmeyi aldığı için cevap üretirken ara adımları açıklamaya çalıştı.

Yani Chain-of-Thought’u şu şekilde kullandık:

Aynı LLM ve tokenizer’ı kullandık, farklı bir yazılım yüklemedik.

Prompt’u “Answer the following question: …” yerine “Answer the following question step by step. Let’s think step by step. Question: …” olarak değiştirdik.

Model bu ilave doğal dil yönlendirmeye dayanarak hem ara hesaplamaları hem de son cevabı metinde açıkladı.

Dolayısıyla Chain-of-Thought, ekstra bir kütüphane değil; doğru prompt’un modelin içindeki öğrenilmiş “düşünme adımlarını” dışa vurmasını sağladığı bir tekniktir.

In [2]:
!pip install transformers sentencepiece --quiet

In [3]:


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Device ayarı: GPU varsa kullan, yoksa CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Cell 2 — Model ve Tokenizer yükleme
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Basit bir örnek problem: "12 + 8 * 3 nedir?"
question = "What is 12 + 8 * 3?"

# Generation ayarları  
gen_kwargs = {
    "max_length": 64,
    "temperature": 0.7,
    "top_p": 0.9,
    "return_dict_in_generate": True,
    "output_scores": True,
}

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

2025-07-12 14:54:00.195675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752332040.565723      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752332040.674219      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
#  Chain-of-thought olmadan cevap
prompt_plain = f"Answer the following question: {question}"
inputs_plain = tokenizer(prompt_plain, return_tensors="pt").to(device)

output_plain = model.generate(**inputs_plain, **gen_kwargs)
answer_plain = tokenizer.decode(output_plain.sequences[0], skip_special_tokens=True)

print("=== Cevap (Chain-of-thought OLMADAN) ===")
print(answer_plain)
print()

# Cell 4 — Chain-of-thought ile cevap
prompt_cot = (
    f"Answer the following question step by step. "
    f"Let's think step by step.\nQuestion: {question}"
)
inputs_cot = tokenizer(prompt_cot, return_tensors="pt").to(device)

output_cot = model.generate(**inputs_cot, **gen_kwargs)
answer_cot = tokenizer.decode(output_cot.sequences[0], skip_special_tokens=True)

print("=== Cevap (Chain-of-thought İLE) ===")
print(answer_cot)
print()

# Cell 5 — Ara çıktılar: Token skorlarından bir kesit
# (sadece örnek olarak ilk birkaç token'un skorlarını gösteriyoruz)
scores = output_cot.scores  # list of logits per generated step
tokens = output_cot.sequences[0]

print("=== Chain-of-thought Ara Çıktı Örneği (DÜZELTİLMİŞ) ===")
for i, score in enumerate(scores[:5]):
    token_id = tokens[i+1].item()    # i+1: ilk üretilen token’ı alıyoruz
    probs = torch.softmax(score, dim=-1)   # shape: [1, vocab_size]
    prob = probs[0, token_id].item()       # önce batch=0, sonra token_id
    token_str = tokenizer.convert_ids_to_tokens(token_id)
    print(f"Adım {i+1}: Token='{token_str}', P={prob:.4f}")
print()

# Cell 6 — Kısa Karşılaştırma
print("=== Karşılaştırma ===")
print(f"- Doğrudan cevap uzunluğu: {len(answer_plain.split())} kelime")
print(f"- COT cevabı uzunluğu:    {len(answer_cot.split())} kelime")
print()
print("Gözlemler:")
print("- Chain-of-thought ile model adımları açıklayarak daha açık ve detaylı bir mantık sunuyor.")
print("- Doğrudan cevap genellikle sadece son sonucu verir; COT ise ara hesaplamaları da içeriyor.")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== Cevap (Chain-of-thought OLMADAN) ===
4

=== Cevap (Chain-of-thought İLE) ===
The number of 3 is 12 + 8 = 24. The number of 12 is 12 + 8 = 24. The answer: 24.

=== Chain-of-thought Ara Çıktı Örneği (DÜZELTİLMİŞ) ===
Adım 1: Token='▁The', P=0.2319
Adım 2: Token='▁number', P=0.3235
Adım 3: Token='▁of', P=0.3922
Adım 4: Token='▁3', P=0.1392
Adım 5: Token='▁is', P=0.4026

=== Karşılaştırma ===
- Doğrudan cevap uzunluğu: 1 kelime
- COT cevabı uzunluğu:    23 kelime

Gözlemler:
- Chain-of-thought ile model adımları açıklayarak daha açık ve detaylı bir mantık sunuyor.
- Doğrudan cevap genellikle sadece son sonucu verir; COT ise ara hesaplamaları da içeriyor.


## Senaryo Bazlı Örnek 2

**Senaryo 1: Standart Yaklaşım ve Başarısızlık**

Prompt: Modele problemi verip sadece nihai sonucu istedik.

Beklenen Sonuç: model, birden çok adımı (3 müşteri * 2 kutu, +15 yeni, -5 imha) aynı anda aklında tutmakta zorlanabilir. Adım adım düşünmeye zorlanmadığı için, işlemleri karıştırabilir veya bir adımı atlayabilir. Bu, genellikle yanlış bir nihai cevapla sonuçlanır.

Gözlem: Modelin çıktısı, CoT olmadan ne kadar kafa karıştırıcı ve güvenilmez olabileceğini gösterir.

**Senaryo 2: CoT'nin Eklenmesi ve Başarı**

CoT Nasıl Ekleniyor?
Yine, prompt'un sonuna sihirli talimatımızı ekledik:
"Lütfen bu problemi adım adım düşünerek çöz. Her adımdaki işlemi ve o adımdan sonraki durumu açıkça belirt."
Bu talimat, modeli bir hesap makinesi gibi davranmaktan çıkarıp, bir problemi çözen bir analist gibi davranmaya teşvik eder.

İlerleme ve Ara Çıktıların İncelenmesi:
Model bu talimatla birlikte şu ara çıktıları üretir:

Başlangıç: "Başlangıçta 20 kutu süt var."

Satışlar: "3 müşteri her biri 2 kutu aldı, toplamda 3 * 2 = 6 kutu satıldı. Kalan süt: 20 - 6 = 14 kutu."

Sevkiyat: "Yeni sevkiyatla 15 kutu geldi. Toplam süt: 14 + 15 = 29 kutu."

İmha: "5 kutu imha edildi. Kalan süt: 29 - 5 = 24 kutu."

Nihai Sonuç: "Gün sonunda markette 24 kutu süt kalır."

**Genel Değerlendirme ve Sağlanan Fayda**

Bu örnek, en önemli noktalardan birini kanıtlıyor: Akıl yürütme teknikleri, sadece en büyük modeller için değildir. Asıl güçleri, daha küçük ve erişilebilir modellerin yeteneklerini maksimize etmektir.

CoT gibi bir teknik kullanarak, devasa bulut sistemlerine ve API anahtarlarına ihtiyaç duymadan, kendi bilgisayarınızda çalışan açık kaynak kodlu bir modelle bile karmaşık, çok adımlı problemleri doğru ve şeffaf bir şekilde çözebilirsiniz. Bu, yapay zekayı daha demokratik, erişilebilir ve anlaşılır hale getiren inanılmaz güçlü bir yaklaşımdır.

In [5]:

# Ortak problem tanımı
problem = (
    "Başlangıçta 20 kutu süt var. "
    "3 müşteri her biri 2 kutu aldı. "
    "Sonra 15 kutu yeni sevkiyat geldi. "
    "Ardından 5 kutu imha edildi. "
    "Gün sonunda kaç kutu süt kalır?"
)

gen_kwargs = {
    "max_length": 128,
    "return_dict_in_generate": True,
    "output_scores": False,
}

# Cell 2 — Senaryo 1: Standart Yaklaşım (Başarısızlık)
prompt_plain = f"Problemi çöz ve sadece nihai sonucu ver: {problem}"
inputs_plain = tokenizer(prompt_plain, return_tensors="pt").to(device)
output_plain = model.generate(**inputs_plain, **gen_kwargs)
answer_plain = tokenizer.decode(output_plain.sequences[0], skip_special_tokens=True)

print("=== Senaryo 1: Standart Yaklaşım ===")
print("Prompt:", prompt_plain)
print("Çıktı (yalnızca sonuç):", answer_plain)
print("Gözlem: Küçük modeller veya kompakt Phi-3 gibi modeller adımları atlayabilir, yanlış sonuç verebilir.\n")

# Cell 3 — Senaryo 2: CoT’nin Eklenmesi ve Başarı
prompt_cot = (
    "Lütfen bu problemi adım adım düşünerek çöz. "
    "Her adımdaki işlemi ve o adımdan sonraki durumu açıkça belirt.\n"
    f"Soru: {problem}"
)
inputs_cot = tokenizer(prompt_cot, return_tensors="pt").to(device)
output_cot = model.generate(**inputs_cot, **gen_kwargs)
answer_cot = tokenizer.decode(output_cot.sequences[0], skip_special_tokens=True)

print("=== Senaryo 2: Chain-of-Thought ile Yaklaşım ===")
print("Prompt:", prompt_cot)
print("Çıktı (adım adım):")
print(answer_cot)
print("\nGözlem: Model tüm ara işlemleri sıraladı ve nihai sonucu doğru buldu.")

# Cell 4 — Kısa Karşılaştırma
print("=== Senaryo Karşılaştırması ===")
print(f"- Standart çıktı uzunluğu: {len(answer_plain.split())} kelime")
print(f"- CoT çıktısı uzunluğu:    {len(answer_cot.split())} kelime")
print("\nÖzet:")
print("Standart prompt yalnızca sonucu verdi; adımları atladığı için hata yapma riski var.")
print("CoT prompt ise her adımı açıkça belirterek doğru ve şeffaf bir sonuç sağladı.")


=== Senaryo 1: Standart Yaklaşım ===
Prompt: Problemi çöz ve sadece nihai sonucu ver: Başlangıçta 20 kutu süt var. 3 müşteri her biri 2 kutu aldı. Sonra 15 kutu yeni sevkiyat geldi. Ardından 5 kutu imha edildi. Gün sonunda kaç kutu süt kalır?
Çıktı (yalnızca sonuç): ki süt süresi için yüzde 20 ve yüzde 3 üyeleri için yüzde 2 süt süresi için yüzde 5 süt süresi için yüzde 5 süt süresi için yüzde 5 süt süresi için yüzde 5
Gözlem: Küçük modeller veya kompakt Phi-3 gibi modeller adımları atlayabilir, yanlış sonuç verebilir.

=== Senaryo 2: Chain-of-Thought ile Yaklaşım ===
Prompt: Lütfen bu problemi adım adım düşünerek çöz. Her adımdaki işlemi ve o adımdan sonraki durumu açıkça belirt.
Soru: Başlangıçta 20 kutu süt var. 3 müşteri her biri 2 kutu aldı. Sonra 15 kutu yeni sevkiyat geldi. Ardından 5 kutu imha edildi. Gün sonunda kaç kutu süt kalır?
Çıktı (adım adım):
Lütfen, yaplan açklamada, yaplan açklamada, yaplan açklamada, yaplan açklamada, yaplan açklamada, yaplan açklamada, yaplan açkla